<a name="top"></a>

# Lesson 4: Stellar Rotation Rates

## Learning objectives<a name="Learningobjectives"></a>
- Extract a stellar rotation rate from a TESS light curve using lightkurve
- Apply the Nyquist-Shannon Samping Theorem when finding stellar rotation rates by restricting periodogram searches
- Assess the level of correlation between two quantities (rotation rate and flare rate)

## Introduction
Stars rotate. This feature is inherently linked to stellar formation — the molecular clouds from which stars form need only very slightly rotate, and conservation of angular momentum makes sure that the final, formed star rotates, as well.

The rotation rate of a star tell us about its evolution, magnetic fields, etc. Stellar light curves (time-series photometry) offer a glimpse into the rotation rates of these bodies. The core idea is that stars are not homogeneous; rather, they can have cooler spots on their surface that rotate in and out of view. The classical picture is of a large single spot that yields a sinusoidal light curve, as an observer detects fewer photons when the spot is in view. In reality, stars are not so ideal; they may have multiple spots at once, and these spots evolve over time. But the general idea remains: the periodicity of a light curve is related to its rotation rate.

Additionally, we expect that faster-rotating stars should exhibit more *flares* — short outbursts of energy greater than baseline levels — because flares and rotation rates should both be connected to the stellar magnetic field. Flares are generally thought to release magnetic energy in stars; in turn, under something known as "dynamo theory," this magnetic energy is tied to the rotation rate of a star (e.g.,  [Moffatt 1978](https://www.researchgate.net/profile/Keith-Moffatt/publication/229086771_The_Generation_of_Magnetic_Fields_in_Electrically_Conducting_Fluids/links/00b7d52c3bea06ba07000000/The-Generation-of-Magnetic-Fields-in-Electrically-Conducting-Fluids.pdf), [Parker 1979](https://adsabs.harvard.edu/full/1979ApJ...234..333P), [Browning 2008](https://iopscience.iop.org/article/10.1086/592397/meta), [Zhang
et al. 2008](https://link.springer.com/article/10.1007/s11207-007-9089-0)). Loosely, we expect that faster-rotating stars should have greater flaring activity, whereas slower-rotating stars should have less flaring activity.

In this notebook, we will work through the key mechanics of identifying a star’s rotation period from its TESS light curve. We will then apply this to a large population of stars to understand how rotation rates vary with flare rate across the M dwarf population. 

## Import Statements<a name="import"></a>

There are many imports in this notebook
* `numpy` is used for array manipulation.
* `matplotlib.pyplot` is used to display images and plot datasets.
* `lightkurve` allows us to easily interact with TESS light curves.
* [stella](https://github.com/afeinstein20/stella) is a purpose-built stellar flare classifier


In [ ]:
# manipulating and plotting arrays
import matplotlib.pyplot as plt
import numpy as np

# file handling, units, and data queries
import tarfile
from tqdm import tqdm
from astropy.io import fits
from astropy.utils.data import download_file
from astroquery.mast import Observations

import astropy
import astropy.constants as const
import astropy.units as u
import lightkurve as lk

# flare-specific machine learning package
import stella

# misc imports
import s3fs
import os
import random

# read from cloud datasets as opposed to downloading from on-premise servers
fs = s3fs.S3FileSystem(anon=True)
Observations.enable_cloud_dataset()

%matplotlib inline

Let's get started by plotting a sample lightcurve and making a periodogram. 

## Plotting a periodic light curve<a name="plotting"> </a>
We'll start by selecting a target with an interesting lightcurve, following the usual syntax.

In [ ]:
# our interesting target
ticid = 234295610

# query for results, filter for light curve files
obs = Observations.query_criteria(target_name=ticid, provenance_name="SPOC", sequence_number=[1])
prod = Observations.get_product_list(obs)
filt = Observations.filter_products(prod, description="Light curves")
filt

We've found our target. Now let's make a plot.

In [ ]:
c_uris = Observations.get_cloud_uris(filt)

# Initialize a TESS lightCurve in lightKurve
with fs.open(c_uris[0], "rb") as f:
    with fits.open(f, "readonly") as fts:
        lc1 = lk.TessLightCurveFile(fts)

# plot the lightcurve
lc1.plot()

This light curve is quite interesting. By eye, there seems to be at least one source of periodicity, and there are a few regions of high flux. Are they flares? We'll return to this question later on by explicitly comparing these features with stellar flare classifiers.

# Finding periodicity in light curves<a name="lightcurveperiodicity"></a>

To address the our first observation - finding periodicity in the data — we need to calculate a *periodogram*. A periodogram is a mathematical estimate of how strong different periods are in a dataset. Under the hood, many periodograms are fundamentally related to <a href="https://news.mit.edu/2009/explained-fourier">Fourier Transforms</a>. Before we call the periodogram function we'll be using, we want to consider what a maximum period of the pulsations might be. Based on some science cases, we might be interested in stellar periodicity that is quite long. Betelgeuse, for instance, is a star that <a href="https://arxiv.org/abs/2306.00287">has quite interesting pulsations</a> on the order of 2200 days. However, we're fundamentally limited by our observing strategy based on something called the Nyquist-Shannon Theorem.

## Relevance of the Nyquist-Shannon Sampling Theorem<a name="nyquist"></a>
The <a href="https://chem.libretexts.org/Ancillary_Materials/Laboratory_Experiments/Wet_Lab_Experiments/Analytical_Chemistry_Labs/ASDL_Labware/Analog_and_Digital_Conversion_for_Chemical_Instrumentation/04_Number_Representation/05_The_Nyquist_Sampling_Theorem">Nyquist theorem</a> is a fundamental aspect of signal processing. It tells us that in order to fully recover a signal at frequency `f`, we must sample at minimum rate of `2f`. The basic intuition is as follows: for a sine wave, our data would need to at least contain the wave hitting both its peak and its trough for us to understand its period. For more information, see [VanderPlas 2018](https://iopscience.iop.org/article/10.3847/1538-4365/aab766#apjsaab766s3), especially section 3.2.1.

What about an upper limit on the period? For simplicity's sake, let's set this to the length of a TESS sector: namely, 28 days.  

Let's implement both of these limits in code.

In [ ]:
max_period = 28*u.day
min_period = 20*u.s

pg = lc1.normalize(unit='ppm').to_periodogram(maximum_period=max_period, minimum_period=min_period)
pg.plot();

We see some strong peaks in this light curve. Let's print out the "best" period (the one corresponding to the strongest periodogram peak) with the below command:

In [ ]:
pg.period_at_max_power

 Let's plot this periodogram on a log scale. What do we see?

In [ ]:
pg.plot(scale='log');

It looks like there's more "noise" at the low-period (high-frequency) end. 

Upon closer inspection, you might notice that the native frequency grid that `lightkurve` uses has narrower spacing at lower periods. Why would this be desirable? Wouldn't we want to evaluate our periodogram on an even period grid?

It turns out that this is a feature, not a bug. Fundamentally, the Lomb-Scargle periodgram will resolve peaks of width $1/P$ at a given period $P$ (e.g., <a href=https://iopscience.iop.org/article/10.3847/1538-4365/aab766/meta>VanderPlas 2018</a>). This means that we need very narrow spacing between periodogram evaluations at low periods, but we're wasting computational resources if we keep that same resolution at higher periods. 

In the commented-out cells below, you can try enforcing an even spacing to confirm you get the same result with more computational expense:

In [ ]:
# frequency_grid = np.linspace(1/max_period, 1/min_period, 100)
# pg_narrow = lc1.normalize(unit='ppm').to_periodogram(maximum_period=max_period, minimum_period=min_period)
# pg_narrow.plot()
# pg_narrow.plot(scale='log')

Looks close enough to the original! Seems like we should stick with the original approach, which saves us computational expense.

## Exercise: Harmonics<a name="exercise2"></a>
Identify the other periods that have relatively high power. Are they harmonics (evenly-spaced multiples) of the 0.762 day period?

Hint: this might be easier to identify in frequency space (i.e. `pg.frequency`.)

# Enforcing a break-up speed limit<a name="breakup"></a>

It seems like there are some very short period peaks in the periodogram. Might they be the stellar rotation rate?

Probably not. We expect the rotation rate of of low-mass stars to be on the order of days to tens of days (e.g., <a href="https://adsabs.harvard.edu/full/record/seri/ApJ../0466/1996ApJ...466..384D.html">Donahue et al. 1996</a>). These shorter-period frequencies may be stellar oscillations or pulsations, or harmonic aliases of a single oscillation/pulsation mode. Aliases are multiples of a "true" periodicity that a periodogram can pick out. For instance, if we observe something with a period of 2 days, the periodogram may also note a strong periodic signals at 4 days, 6 days, etc. because the 2 day cycle also coincides with those other cycles.

We can rule these out as stellar rotation rates that would imply that the star is rotating at twice the breakup speed of a main-sequence star. Let's first calculate the break-up rotational period of the Sun. A simplistic approach, ignoring for instance radiation pressure and the changing geometry of a star at higher rotation rates (i.e., bulging at the equator), finds this relationship by setting the equatorial centrifugal acceleration and acceleration by gravity to be equal. A more complex approach (see, e.g., <a href="https://www.astro.umd.edu/~jph/Stellar_Rotation.pdf">these notes</a>) gets an extra constant factor of 2/3 for that breakup speed.

In [ ]:
G = const.G
M = 1 * u.M_sun
R = 1 * u.R_sun
vc = ((2/3) * G * M / R)**.5

P = 2 * np.pi * R  / vc

# print out the result in days
P.to(u.day)

Let's generously assume that a star in our sample could rotate 100 times faster than this without breaking up. In doing so, we're letting `R*M` independently vary by a factor of 10,000. Given what we know about mass-radius relationships of stars (thinking about, e.g., <a href="https://astro.unl.edu/naap/hr/hr_background3.html">Hertzsprung-Russell diagrams</a>), this is at least a plausible physical assumption on the stellar main sequence. Broadly, we're making the statement that

$P_{\rm breakup} \propto \sqrt{RM}$.

This relation sets a new minimum period that's longer than our sampling rate. 

Note that this approach works well enough, but it is not precise enough for publishable research. For a manuscript, other caveats would need to be assessed — e.g., explicitly checking the stellar type of the object we are examining.

In [ ]:
# allow for other types of stars
min_period = P/100
pg = lc1.normalize(unit='ppm').to_periodogram(maximum_period=max_period, minimum_period=min_period)

pg.plot(scale='log');

Great! It looks like our strongest signal is now clearly the peak just shy of 1 day.

It's still possible that the peaks in the periodogram to not correspond to actual stellar rotation rates. One other confounding source of astrophysical periodicity could be binary stars. It's out of scope for this notebook, but it'd be possible to check whether the the lightcurve's variability is due to the presence of a binary star by checking the *morphology* of the periodic signal. Basically, (eclipsing) binary stars will have periodic [V shapes in their lightcurves](https://docs.lightkurve.org/tutorials/3-science-examples/periodograms-creating-periodograms.html), whereas lightcurves that are periodic because of stellar rotation will have a [periodic signal closer to a sine wave](https://docs.lightkurve.org/tutorials/3-science-examples/periodograms-measuring-a-rotation-period.html). Binary stars with [ellipsoidal variations due to strong tidal forces](https://en.wikipedia.org/wiki/Rotating_ellipsoidal_variable) may have rotation signals that are indeed closer to a sine wave; the curious should revisit our [astereoseismology lesson](../02-astroseismology/02-astroseismology.ipynb).

# Rotation rates of a larger population<a name="rotation_pop"></a>
Now, let's wrap the above in a function so that we can run it on a number of different objects.

In [ ]:
def calc_rotation_rate(name):
    """
    Calculates the roation period of a star, assuming that rotation is the primary driver of the star's periodicity. 
    
    Inputs
    ------
        :name: name of star (str)
        
    Outputs
    -------
        :period_at_max_power: the dominant period in the light curve. [float]
    """
    # query for file
    obs = Observations.query_criteria(target_name=name, provenance_name="SPOC", sequence_number=[1, 2])
    prod = Observations.get_product_list(obs)
    filt = Observations.filter_products(prod, description="Light curves")
    c_uris = Observations.get_cloud_uris(filt)
    
    # open in lightkurve
    with fs.open(c_uris[0], "rb") as f:
        with fits.open(f, "readonly") as fts:
            lc = lk.TessLightCurveFile(fts)
    
    pg = lc.normalize(unit='ppm').to_periodogram(maximum_period=max_period, minimum_period=min_period)
    
    return pg.period_at_max_power

Let's use a population of stars from a [different notebook](https://spacetelescope.github.io/hellouniverse/notebooks/hello-universe/Classifying_TESS_flares_with_CNNs/Classifying_TESS_flares_with_CNNs.html).

In [ ]:
file_url = 'https://archive.stsci.edu/hlsps/hellouniverse/hellouniverse_stella_500.tar.gz'
file = tarfile.open(download_file(file_url, cache=True))
file.extractall('.')
file.close() # be sure to close files when you're finished with them!

With the file now downloaded, we can use the `astropy` package to read in the flare catalog file.

In [ ]:
data_dir = './hellouniverse_stella_500/'
filename = 'Guenther_2020_flare_catalog.txt'

catalog = astropy.io.ascii.read(data_dir + filename)  
catalog

We're just interested in the stars themselves, which we can access with the following syntax:

In [ ]:
stars = catalog['TIC'].data
stars

Now, we apply this to the population. The whole population is quite large — if we use only 200th star, then the calculation will only take less than a couple of minutes.

In [ ]:
rot_rates = []
for star in tqdm(stars[::1000]):
    rot_rate = calc_rotation_rate(str(star))
    
    # store result of rotation rate calculation
    rot_rates += [rot_rate]

We can use a histogram to get a sense of how the periods are distributed.

In [ ]:
bins=np.linspace(0,14,14)
plt.hist(rot_rates, color='teal',bins=bins)
plt.xlabel('Period (days)', fontsize=25)
plt.ylabel('Count', fontsize=25)

In the above plot, we see a broad distribution of periodicities. Most of the results cluster around 0-1 days, but we see some longer-period objects as well.

# Assessing flares<a name="flares"></a>
In a different notebook, we trained a convolutional neural network (CNN) to classify stellar flares using the `stella` package (<a href="https://joss.theoj.org/papers/10.21105/joss.02347.pdf">Feinstein et al. 2020</a>). 

We can use the default code to answer the following question: Does a star's rotation rate affect its flaring rate?

In [ ]:
# set up the ConvNN object.
cnn_stella = stella.ConvNN(output_dir=data_dir)

Next, let's check that our classifier is working correctly. To do so, we can just apply it to the first light curve we assessed above.

In [ ]:
cnn_stella.predict(modelname="initial_flare_model_small_dataset.h5",
                   times=lc1.time.value, fluxes=lc1.flux, errs=lc1.flux_err)

Our classifier has run, but now we need to see the output! Let's make a plot.

In [ ]:
times = cnn_stella.predict_time[0]
fluxes = cnn_stella.predict_flux[0]
prediction = cnn_stella.predictions[0]

# Create a scatterplot
fig, ax = plt.subplots()
im  = ax.scatter(times, fluxes, c=prediction, s=2, cmap="PuOr")

# add some labels
plt.xlabel('Time (d)')
plt.ylabel('Flux (normalized)')
plt.colorbar(im, label='Flare probability')
plt.show()

This result seems to make sense. The "peakiest" parts of the light curves are labeled as flares by our classifier at high probability. 

To further verify this, we can zoom in and pick out some of the flare morphology:

In [ ]:
# create another plot
fig, ax= plt.subplots()
im  = ax.scatter(times, fluxes, c=prediction, s=5, cmap="PuOr")

# add labels
plt.xlabel('Time (d)')
plt.ylabel('Flux (normalized)')
plt.colorbar(im, label='Flare probability')

# zoom in
plt.xlim(min(times), 1326)
plt.show()

We see a sharp increase in the flux followed by a slower fall — an example of classic flare morphology (e.g., <a href=https://iopscience.iop.org/article/10.3847/1538-3881/ac6fe6/meta>Mendoza et al. 2022</a>). Looks like the classifier is working!

How do we next assess the flare *rate*, now that we've identified flares in our light curve? The simplest thing to do here (and the thing we will start with) is to calculate what fraction of the time is spent flaring. Whlie this may overemphasize the presence of large / long flares, this metric is a good starting point.

In [ ]:
flare_cutoff = 0.5 # the minimum value above which the CNN classifies a flare

total_length = len(cnn_stella.predictions[0])
flaring_length = len(cnn_stella.predictions[0][cnn_stella.predictions[0] > flare_cutoff])
frac_flaring = flaring_length/total_length

Let's create a plot of this.

In [ ]:
plt.scatter(pg.period_at_max_power, frac_flaring)

plt.xlabel('Period at max power (d)')
plt.ylabel('Fraction of time spent flaring')

# Checking flares for the whole population<a name="flares_pop"></a>

So far, we've just calculated a single data point! The next step is to do this for the whole  population.

In [ ]:
def calc_flare_rate(name):
    """
    Calculates the flare rate of a light curve based on how much time it spends flaring vs. not flaring.
    
    Inputs
    ------
        :name: (str) TICID of star.
        
    Outputs
    ------------
        :flare_rate: (float) flare rate.
    """
    # query for file
    obs = Observations.query_criteria(target_name=name, provenance_name="SPOC", sequence_number=[1, 2])
    prod = Observations.get_product_list(obs)
    filt = Observations.filter_products(prod, description="Light curves")
    c_uris = Observations.get_cloud_uris(filt)
    
    # open in lightkurve
    with fs.open(c_uris[0], "rb") as f:
        with fits.open(f, "readonly") as fts:
            lc = lk.TessLightCurveFile(fts)


    cnn_stella.predict(modelname="initial_flare_model_small_dataset.h5", times=lc.time.value, fluxes=lc.flux, errs=lc.flux_err)
    
    flare_rate = len(cnn_stella.predictions[0][cnn_stella.predictions[0] > 0.5])/len(cnn_stella.predictions[0])
    
    return flare_rate

Calculating the flare rate for the whole population will take quite a while. For the purposes of this tutorial (i.e, we'll only look at every two hundredth star.

We won't use the `tqdm` package this time, because it will produce a lot of output in combination with the `stella` predictions.

In [ ]:
%%time

flare_rates = []

# go through the list of targets
for star in stars[::1000]:
    flare_rate = calc_flare_rate(str(star))
    flare_rates += [flare_rate]

We first examine our flare rate alone.

In [ ]:
plt.hist(flare_rates, color='goldenrod')
plt.xlabel('Flare rate', fontsize=25)
plt.ylabel('Count', fontsize=25)

# Comparing flare rate and rotation rate<a name="compare_rates"></a>
At this point, we have measurements of two quantities — flare rate and rotation rate — that we trust well enough. We can now ask the final question: Are these two quantities related in our sample? Recall that we expect faster-spinning stars to flare more frequently.

Let's see whether this expected trend bears out in our data.

In [ ]:
rot_rates = [rot_rate.value for rot_rate in rot_rates]

In [ ]:
plt.scatter(rot_rates, flare_rates, color='teal')

plt.xlabel('Period at max power (d)')
plt.ylabel('Fraction of time spent flaring')

Many astrophysical processes obey log-linear scaling relationships. That is, when we plot some relationships on log-log axes, they look like straight lines. Let's see what happens when we plot our data on a log-log plot.

In [ ]:
plt.scatter(rot_rates, flare_rates, color='teal')

plt.xlabel('Period at max power (d)')
plt.ylabel('Fraction of time spent flaring')
plt.yscale('log')
plt.xscale('log')

From this plot, it appears that shorter-period stars spend more time flaring! Let's quantify this statistically.

To do so, let's perform linear regression on our data. Let's start by cleaning out any 

In [ ]:
# making our results into arrays so we can easily mask them
rot_rates, flare_rates = np.array(rot_rates), np.array(flare_rates)

# mask out unrealistic flare rates
rot_rates_cleaned = rot_rates[flare_rates>0.0]
flare_rates_cleaned = flare_rates[flare_rates>0.0]

In [ ]:
# perform regression on the log values
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(np.log10(rot_rates_cleaned), np.log10(flare_rates_cleaned))

print(slope, intercept)

In [ ]:
x = np.linspace(np.min(np.log10(rot_rates_cleaned)), np.max(np.log10(rot_rates_cleaned)), 100)
y = slope * x + intercept

In [ ]:
plt.scatter(np.log10(rot_rates_cleaned), np.log10(flare_rates_cleaned), color='teal', label='TESS data')
plt.plot(x,y, color='black', linestyle='--', lw=4, label='Regression best fit')
            
plt.ylabel(r'$\log_{10}$' + ' (Fraction of time spent flaring)')
plt.xlabel(r'$\log_{10}$' + ' (Period at max power (d))')
plt.legend(fontsize=14);

This is quite suggestive of the overall trend we predicted: that faster-rotating stars would have higher flaring activity. It may be tempting to call it a day, but there's still more due diligence to perform before we can claim that we've verified a prediction. In exercises 4-6, we assess whether our relationship is robust and statistically significant.

One final note: for stars with rotational periods < 0.5 days, stella may misclassify variability due to rotation as flaring activity. To get a rough sense of how this affects our results, let's see where that rotation rate threshold lies on our plot.

In [ ]:
plt.scatter(np.log10(rot_rates_cleaned),np.log10(flare_rates_cleaned), color='teal', label='TESS data')
plt.plot(x,y, color='black', linestyle='--', lw=4, label='Regression best fit')

plt.axvline(np.log10(0.5), color='gray', lw=3, label='stella threshold')
            
plt.ylabel(r'$\log_{10}$' + ' (Fraction of time spent flaring)')
plt.xlabel(r'$\log_{10}$' + ' (Period at max power (d))')
plt.legend(fontsize=14);

Hmm, it seems like a decent number of our points are affected. Let's mask out these points and fit our line again.

In [ ]:
# mask out the suspect values
rot_rates_cleaned_again = rot_rates_cleaned[rot_rates_cleaned>0.5]
flare_rates_cleaned_again = flare_rates_cleaned[rot_rates_cleaned>0.5]

# perform regression on the log values
slope_cleaned_again, intercept_cleaned_again, r_value_cleaned_again, p_value_cleaned_again, std_err_cleaned_again = scipy.stats.linregress(np.log10(flare_rates_cleaned_again), 
                                                                     np.log10(rot_rates_cleaned_again))

# create the plot, again: copy/paste from above
x = np.linspace(np.min(np.log10(rot_rates_cleaned_again)), np.max(np.log10(rot_rates_cleaned_again)), 100)
y = slope * x + intercept

plt.scatter(np.log10(rot_rates_cleaned_again),np.log10(flare_rates_cleaned_again), color='teal', label='TESS data')
plt.plot(x,y, color='black', linestyle='--', lw=4, label='Regression best fit')
            
plt.ylabel(r'$\log_{10}$' + ' (Fraction of time spent flaring)')
plt.xlabel(r'$\log_{10}$' + ' (Period at max power (d))')
plt.legend(fontsize=14);

print(slope_cleaned_again, intercept_cleaned_again), print(slope, intercept)

The calculated slope is a little shallower, but it's still negative and generally fits well with our data.

## Next time on MAST Summer Webinar
We'll dive a bit deeper into the statistics of this relationship. Stay tuned!

In [ ]:
# save these values for the next notebook!
np.savetxt("rot_rates_constrained.txt", rot_rates_cleaned_again)
np.savetxt("flare_rates_durations.txt", flare_rates_cleaned_again)

# Additional exercises<a name="more_exercises"></a>
Note that the solutions to these exercises are involved, but much of the code is reproducing work from this notebook (e.g., calculating the break-up rotation speed).

1. There are more robust ways to determine flare rate. For instance, instead of dividing in-flare time by total flare time, we can assess the number of flares per day. Plot the number of flares per day against the rotation rate — does the functional form of this relationship look different from the plot generated in this notebook?
2. In Feinstein et al. 2020, the authors describe three criteria that define robust rotation rates. Implement the below, being sure to mask out harmonics before identifying the secondary peak. Do these steps change your answers? HINT: the `stella` package may include some <a href="https://adina.feinste.in/stella/getting_started/other_features.html">useful functionality</a>.
    - <i>the rotation rate must be less than 12 days</i>
    - <i>the width of a Gaussian fit to the peak power must be less than 40% of the peak period</i>
    - <i>the secondary peak in the power spectrum must be 4% weaker than the primary peak</i>
3. Feinstein et al. 2020 also identified the below checks for verifying flares. Implement these, as well; does this impact the flares vs. rotation rate plot? This exercise can be performed independently of Exercise 5.
    - <i>the amplitude of the flare must be 1.5σ > the locally detrended light curve</i>
    - <i>the two cadences directly following the amplitude must be 1σ > the detrended light curve (i.e., at least three consecutive outlier points are considered part of a flare);</i>
    - <i>the cadence before and after the flare amplitude must be less than the amplitude;</i>
    - <i>the amplitude must be >0.5%.</i>  

# Resources<a name="resources"></a>
- <a href=https://github.com/afeinstein20/stella> stella GitHub repository </a>
- <a href=https://adina.feinste.in/stella/> stella documentation </a>
- <a href=https://adsabs.harvard.edu/full/2004IAUS..215...57G>review on stellar rotation in clusters</a>
- <a href=https://iopscience.iop.org/article/10.3847/1538-4365/aab766/meta>VanderPlas 2018: Understanding the Lomb–Scargle Periodogram</a>
- <a href=https://online.stat.psu.edu/stat510/lesson/6/6.1#:~:texta%20time%20series.-,Periodogram,encountered%20monthly%20or%20quarterly%20seasonality.>Introduction to periodograms</a>
<a href=https://en.wikipedia.org/wiki/Nyquist%E2%80%93Shannon_sampling_theorem>Wikipedia article on the Nyquist-Shannon sampling theorem</a>

## About this Notebook

**Author:** Arjun Savel

**Last updated:** June 2024 <br>

For support, please contact the Archive HelpDesk at archive@stsci.edu.

***

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>

[Return to top of page](#top)